In [91]:
import pandas as pd 

In [92]:
data = pd.read_csv('data\prix-carburants-fichier-quotidien-test-ods.csv', sep= ';', on_bad_lines= 'skip')

In [93]:
data.columns

Index(['id', 'cp', 'pop', 'adresse', 'ville', 'horaires', 'rupture',
       'fermeture', 'geom', 'prix_maj', 'prix_id', 'prix_valeur', 'prix_nom',
       'com_arm_code', 'Nom Officiel Commune / Arrondissement Municipal',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel Département',
       'Nom Officiel Département', 'Code Officiel Région',
       'Nom Officiel Région', 'services_service', 'Nom rupture',
       'Début rupture', 'Fin rupture', 'Automate 24-24 (oui/non)'],
      dtype='object')

Fonction pour définir la "bonne" base de données avec les données qui nous intéressent  
Il faut l'améliorer pour qu'elle élimine les doublons

In [94]:
def data_carb(df, carb):
    data_1 = df[['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom']]
    d = data_1[data_1.prix_nom == carb]
    d=d.drop_duplicates(keep = 'first', inplace=True) 
    #df.drop_duplicates(subset ="'column'", keep = 'first', inplace=True) si on veut supprimer selon une colonne

    return d




In [95]:
data_2 = data_carb(data, 'SP95').sort_values('prix_valeur')

In [96]:
data_2

,adresse,ville,geom,prix_valeur,prix_nom
2601,Avenue de l'Orme à Martin,Courcouronnes,"48.631,2.417",1.169,SP95
67001,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
2737,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
38557,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
65471,13 Avenue Ampère,MONTIGNY-LE-BRETONNEUX,"48.798,2.048",1.423,SP95
...,...,...,...,...,...
46107,15 Avenue Duquesne,PARIS,"48.853,2.308",2.600,SP95
53592,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95
58108,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95
43085,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95


Fonction pour calculer la distance à partir de données de longitude et de latitude

In [97]:
from math import cos, sin, acos 

def distance1(p1,p2):
    lat1,long1 = p1
    lat2,long2 = p2
    dlong = long2 - long1
    dist_ang = acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(dlong))
    return(6378137*dist_ang)

In [100]:
from geopy.geocoders import Nominatim


def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="Projet Python")
    location = geolocator.geocode(Adresse)
    return str(str(location.latitude) + ','+ str(location.longitude))


Adresse_to_gps("1 rue d'Arsonval, Orsay, France")

'48.7087952,2.1742033'

Fonction pour renvoyer un data frame avec les 10 plus proches stations essences qui propose le type de carburant recherché   
Majeur problème : les doublons 

In [101]:
def dix_plus_proche_card(df, posi, carb):
    data_1 = data_carb(df, carb).sort_values('prix_valeur')
    pos = list(map(float,Adresse_to_gps(posi).split(',')))
    def distancetoposi(p2):
        pos_carb = list(map(float, p2.split(',')))
        return distance1(pos, pos_carb)
    new_df = data_1['geom'].apply(distancetoposi).sort_values().head(10)
    return data_1.loc[new_df.index.to_list()]



    


In [103]:
dix_plus_proche_card(data, "1 rue d'Arsonval, Orsay, France" , 'E10')

,adresse,ville,geom,prix_valeur,prix_nom
68902,Route de Longjumeau,CHILLY-MAZARIN,"48.71,2.298",1.750,E10
55130,Route de Longjumeau,CHILLY-MAZARIN,"48.71,2.298",1.750,E10
76068,69 Avenue Charles de Gaulle,MORANGIS,"48.706,2.343",1.709,E10
25262,69 Avenue Charles de Gaulle,MORANGIS,"48.706,2.343",1.709,E10
13691,38 AV DE MORANGIS,ATHIS MONS,"48.706,2.369",1.859,E10
41567,CHEMIN DE BRIIS CD 59,Villebon-sur-Yvette,"48.7046,2.25396",1.659,E10
28596,CHEMIN DE BRIIS CD 59,Villebon-sur-Yvette,"48.7046,2.25396",1.659,E10
45188,CHEMIN DE BRIIS CD 59,Villebon-sur-Yvette,"48.7046,2.25396",1.659,E10
41573,180 ROUTE NATIONALE 7,ATHIS-MONS,"48.712,2.371",1.656,E10
61582,RUE DE LONGJUMEAU RUE AMPERE,Chilly-Mazarin,"48.713,2.298",1.889,E10
